In [42]:
import win32com.client
import pandas as pd
import datetime
import re
import numpy as np

In [2]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

In [3]:
inbox = outlook.GetDefaultFolder(6) #6 is the inbox, can change it for other folders

In [4]:
#see all the folders that are available
for folder in outlook.Folders: 
    print(folder.Name)

Public Folders - isabel.fernandez@maastrichtuniversity.nl
assistant-iberialatam@maastrichtuniversity.nl
Public Folders - info-iberialatam@maastrichtuniversity.nl
Public Folders - assistant-iberialatam@maastrichtuniversity.nl
info-iberialatam@maastrichtuniversity.nl
Outlook Data File
i.fernandezpereira@student.maastrichtuniversity.nl
Public Folders - i.fernandezpereira@student.maastrichtuniversity.nl
isabel.fernandez@maastrichtuniversity.nl


In [25]:
if 1==0: #make this true when you want the main inbox
    messages = inbox.Items
    
else: #use this when trying to access another folder or a delegated mailbox
    
    root_folder = outlook.Folders.Item(7) #this allows you to access different inboxes 
    #make sure correct inbox is being used
    print(root_folder)
    subfolder = root_folder.Folders['Inbox']
    #get emails from that inbox
    messages = subfolder.Items
    
#sort messages by the time they were received
messages.Sort("ReceivedTime")
message = messages.GetFirst()
print(message)

#attach = message.Attachments
#print (body_content)
#print(datetime)
#print(sender)
#print(subject)

i.fernandezpereira@student.maastrichtuniversity.nl
Participate in a promotional video for the Week of the International Student!


In [26]:
#create empty dataframe, fill it in as emails are read
df= pd.DataFrame(columns=['subject', 'sender', 'date', 'content of message'])

In [27]:
#first cast the datetime to string, so that it can be stripped of the timezone and used more easily
#The error correction can propably be improved
def get_date (date, i):
    date= str(date)
    try:
        date = datetime.datetime.strptime(date.rstrip("+00:00"), '%Y-%m-%d %H:%M:%S')
    except ValueError:
        date = datetime.datetime.strptime(date.rstrip("+00:00"), '%Y-%m-%d %H:%M')
    df.loc[i,'date'] =date
#read body firt, remove unwanted text, then feed it to the df
def get_content (body, i):
    body= body.replace("\r"," ") 
    body= body.replace("\n","") 
    body= body.replace("\t","") 
    df.loc[i,'content of message'] = body

In [28]:
for i in range(0,50):
    try:
        df.loc[i,'subject']=message.Subject
        df.loc[i,'sender']=message.Sender
        get_date(message.CreationTime, i)      
        get_content(message.body, i)
        
    except AttributeError as e:
        if e == 'GetNext.Subject':
            df.loc[i,'subject']= None
            df.loc[i,'sender'] = message.Sender
            get_date(message.CreationTime, i)
            get_content(message.body, i)
        elif e == 'GetNext.Sender':
            df.loc[i,'sender'] = None
            get_date(message.CreationTime, i)
            get_content(message.body, i)
        elif e=='GetNext.message.body':
            df.loc[i,'content of message'] = None
           
            
    message= messages.GetNext()



In [29]:
display(df)

,subject,sender,date,content of message
0,Participate in a promotional video for the Wee...,"Karageorgieva, Radostina (DKE)",2017-10-31 12:19:26,"Dear students, For the upcoming Week of t..."
1,FW: Kick-off event --> Your supervisor cannot ...,honoursplus (EDLAB),2017-10-31 14:56:32,"Dear students, Unfortunately, your Supervi..."
2,UM News,Maastricht University,2017-10-31 15:16:18,31-10-2017 View this <http://mailing.maastric...
3,KE@Work: info session tomorrow,Kework (DKE),2017-11-01 09:53:04,"Dear all, Further to your questions yester..."
4,H+: thank you all for last night,honoursplus (EDLAB),2017-11-01 10:50:49,"Dear Honours+ students, Thank you for a g..."
5,Let us show you the light! Come and pick up yo...,"Karageorgieva, Radostina (DKE)",2017-11-01 15:43:03,"Dear students, On behalf of DKE's staff, ..."
6,date reminder - for Anna & Guus,"Wyatt, Sally (TSS)",2017-11-01 17:47:43,"Dear all, Sounds like the kick-off meetin..."
7,Accepted: Meeting on Electives/Minor,NaN,NaN,NaN
8,"Invitation DKE Lunch lecture: Wednesday, Novem...","Narinx, Ellen (DKE)",2017-11-02 11:08:24,"Dear all, You are kindly invited to our fi..."
9,Last ISAP Intervision,Info-isap (SSC),2017-11-02 12:59:02,"Dear Ambassador, This is a reminder to at..."


In [30]:
#change file name accordingly 
df.to_csv('.\dataset\Delegated_Email.csv')

In [100]:
from Parser import read_email, corpus2sentences, generate_text
from spacy.lang.en import English
def num_question(email):
    counter=0
    for char in email:
        if char == '?':
            counter+=1
    
    return counter

#TODO: create a function to extract questions
#def get_questions(email):
    

#TODO: make it all lowercase
#super basic function, more research needed
def remove_signature(email):
    sep= ''
    #create list of words that show end of email
    end_keywords= ['regards,', 'best,' ]
    for word in email.split():
        if word in end_keywords:
            sep = word
            #remove everything after end keyword
            email = email.split(sep, 1)[0]
    return email
        
def remove_unwanted(email):
    emails = re.findall(r'[\w\.-]+@[\w\.-]+', email)
    websites= re.findall('<?(?:http[s]?://|www\.)[\w.-/]+>?', email)
    print(websites)
   # emails.append(re.findall())
    for word in email.split():
        if word in emails:
            email=email.replace(word, '')
        elif word in websites:
            email=email.replace(word, '')
    return email

    
        

In [101]:
string =df.loc[40,'content of message']
string= string.lower()
print(string)

dear all,      on november 30th we organize an edition of tafelen together with the innbetween <https://www.facebook.com/intheinnbetween/> ! tafelen is their weekly dinner where they host students for a three course dinner and always aim to have a good conversation. whether it’s a political debate, a religious discussion or just an amble chat, conversation over the dinner table is an important thing. you can have a 3 course dinner, cooked by volunteers for only 3,50!    — registration — ticket page <http://innbetween.nl/events/tafelen-weekly-dinner-57/> . registration is required! this is so we know how many people to cook for. you can pay online via ideal or with mastercard/visa. online payment is €3.50 whereas payment at the door will cost €5, so make sure you pay online! if you do not register and would still like to come, the meal will cost €7.50 [only if there is enough food].     — volunteers — volunteers needed! if you want to help out with the cooking, please sign up via the ti

In [102]:
num_question(string)

1

In [103]:
string =remove_signature(string)
print(string)

dear all,      on november 30th we organize an edition of tafelen together with the innbetween <https://www.facebook.com/intheinnbetween/> ! tafelen is their weekly dinner where they host students for a three course dinner and always aim to have a good conversation. whether it’s a political debate, a religious discussion or just an amble chat, conversation over the dinner table is an important thing. you can have a 3 course dinner, cooked by volunteers for only 3,50!    — registration — ticket page <http://innbetween.nl/events/tafelen-weekly-dinner-57/> . registration is required! this is so we know how many people to cook for. you can pay online via ideal or with mastercard/visa. online payment is €3.50 whereas payment at the door will cost €5, so make sure you pay online! if you do not register and would still like to come, the meal will cost €7.50 [only if there is enough food].     — volunteers — volunteers needed! if you want to help out with the cooking, please sign up via the ti

In [104]:
#string= 'Hello world! www.this.com and also try this thing http://www.test.com.br'
string =remove_unwanted(string)
print(string)

['<https://www.facebook.com/intheinnbetween/>', '<http://innbetween.nl/events/tafelen', '<https://www.facebook.com/events/336153800182191/>']
dear all,      on november 30th we organize an edition of tafelen together with the innbetween  ! tafelen is their weekly dinner where they host students for a three course dinner and always aim to have a good conversation. whether it’s a political debate, a religious discussion or just an amble chat, conversation over the dinner table is an important thing. you can have a 3 course dinner, cooked by volunteers for only 3,50!    — registration — ticket page <http://innbetween.nl/events/tafelen-weekly-dinner-57/> . registration is required! this is so we know how many people to cook for. you can pay online via ideal or with mastercard/visa. online payment is €3.50 whereas payment at the door will cost €5, so make sure you pay online! if you do not register and would still like to come, the meal will cost €7.50 [only if there is enough food].     — 

In [112]:
#create Bag of Words
import numpy as np
import re


In [158]:
def extract_words(text):
    ignore_words=['and', 'a', 'the', 'regards']
    words = re.sub("[^\w]", " ",  text).split()
    words_cleaned = [w.lower() for w in words if w not in ignore_words]
    return words_cleaned 

def tokenize_sentences(documents):
    words = []
    for text in documents:
        w = extract_words(text)
        words.extend(w)
        
    words = sorted(list(set(words)))
    return words

#input: collection of documents and sentece to be vectorized
def bag_of_words(documents, sentence):
    vocab = tokenize_sentences(documents) #get vocabulary of all documents
    sentence_words = extract_words(sentence) #get words on specific sentence
    bag = np.zeros(len(vocab))
    for sent_word in sentence_words:
        for i,word in enumerate(vocab):
            if sent_word == word:
                bag[i] += 1
                
    return np.array(bag)

def get_vocab(document):
    vocab= sorted(list(set(extract_words(document))))
    return vocab
    

In [159]:
se ='Hello, world! How are you? This is Isabel speaking, hello'
sa = 'Hello! This is another string, which I am using for testing purposes. Hello world'

test = se, sa

In [160]:
bag_of_words(test, sa)

array([1., 1., 0., 1., 2., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1.,
       0.])

In [172]:
#implement tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

In [175]:
vectorizer = TfidfVectorizer()
response = vectorizer.fit_transform(test)
print(response)

  (0, 4)	0.48684053853849035
  (0, 15)	0.24342026926924518
  (0, 5)	0.34211869506421816
  (0, 2)	0.34211869506421816
  (0, 16)	0.34211869506421816
  (0, 12)	0.24342026926924518
  (0, 6)	0.24342026926924518
  (0, 7)	0.34211869506421816
  (0, 9)	0.34211869506421816
  (1, 4)	0.41882950152188886
  (1, 15)	0.20941475076094443
  (1, 12)	0.20941475076094443
  (1, 6)	0.20941475076094443
  (1, 1)	0.2943251253176751
  (1, 10)	0.2943251253176751
  (1, 14)	0.2943251253176751
  (1, 0)	0.2943251253176751
  (1, 13)	0.2943251253176751
  (1, 3)	0.2943251253176751
  (1, 11)	0.2943251253176751
  (1, 8)	0.2943251253176751
